In [3]:
# Demo of perceptron algorithm in 2d
# Code is modified from
# (2022) Murphy - Probabilistic machine learning [https://probml.github.io/pml-book/book1.html]


import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Perceptron
from sklearn.datasets import load_digits
digits = load_digits()
from copy import copy # technical, for copying artists in plots

def generateData(n):
    # Generiert Daten in 2D mit 2n Datenpunkten, die mit einer Gerade getrennt werden können
    # Blau oben links und gelb unten rechts.
    xb = (np.random.rand(n) * 2 - 1) / 2 - 0.5
    yb = (np.random.rand(n) * 2 - 1) / 2 + 0.5
    xr = (np.random.rand(n) * 2 - 1) / 2 + 0.5
    yr = (np.random.rand(n) * 2 - 1) / 2 - 0.5
    XB = np.stack([xb, yb], axis=1)
    XR = np.stack([xr, yr], axis=1)
    X = np.concatenate([XB, XR])
    #y = np.concatenate([np.zeros(n, dtype=np.int), np.ones(n, dtype=np.int)])
    y = np.concatenate([np.zeros(n, dtype='int'), np.ones(n, dtype='int')])
    return X, y

def plot_dboundary_contourf(net):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    h = 0.02
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    fig, ax = plt.subplots()
    Z = net.predict(np.c_[xx.ravel(), yy.ravel()])

    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, cmap=plt.cm.Paired)  # 
    #ax.axis("off")
    ax.set_xticks([])
    ax.set_yticks([])
    colormap = np.array([plt.cm.Paired(0.1), plt.cm.Paired(0.9)])
    ax.scatter(X[:, 0], X[:, 1], c=colormap[y])
    ax.set_xlabel('$x_1$')
    ax.set_ylabel('$x_2$')
    # ax.axis('square')
    

def plot_dboundary_contourf_ax(net):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    h = 0.02
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    #fig, ax = plt.subplots()
    Z = net.predict(np.c_[xx.ravel(), yy.ravel()])

    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, cmap=plt.cm.Paired)  # 
    #ax.axis("off")
    ax.set_xticks([])
    ax.set_yticks([])
    colormap = np.array([plt.cm.Paired(0.1), plt.cm.Paired(0.9)])
    ax.scatter(X[:, 0], X[:, 1], c=colormap[y])
    ax.set_xlabel('$x_1$')
    ax.set_ylabel('$x_2$')
    # ax.axis('square')


def plot_dboundary(weights, offset):
    w1 = weights[0]
    w2 = weights[1]
    b = offset
    slope = -w1 / w2
    intercept = -b / w2
    xx = np.linspace(-1, 1, 10)
    yy = xx * slope + intercept
    plt.figure()
    colormap = np.array(["r", "k"])
    plt.scatter(X[:, 0], X[:, 1], c=colormap[y], s=40)
    plt.plot(xx, yy, "k-")


def plot_neuron_schematic():
    pos_in_w1    = (0,1)
    pos_in_w2    = (0,0)
    pos_in_bias  = (0,2)
    pos_out      = (2,1)
    neuron_in_w1   = plt.Circle(pos_in_w1,   0.2, color='y')
    neuron_in_w2   = plt.Circle(pos_in_w2,   0.2, color='y')
    neuron_in_bias = plt.Circle(pos_in_bias, 0.2, color='b')
    neuron_out     = plt.Circle(pos_out,     0.2, color='g')
    neuron_list = [neuron_in_w1, neuron_in_w2, neuron_in_bias, neuron_out]

    for n in neuron_list:
        neuron = copy(n)
        ax.add_patch(neuron)

    ax.plot([pos_in_w1[0],   pos_out[0]], [pos_in_w1[1],   pos_out[1]], lw = w1 * 5, zorder=0)
    ax.plot([pos_in_w2[0],   pos_out[0]], [pos_in_w2[1],   pos_out[1]], lw = w2 * 5, zorder=0)
    ax.plot([pos_in_bias[0], pos_out[0]], [pos_in_bias[1], pos_out[1]], lw = b  * 5, zorder=0)

    xy = ((pos_in_bias[0] + pos_out[0])/2 - 0.2,(pos_in_bias[1] + pos_out[1])/2)
    t = ax.annotate('$b$', xy=xy)
    t.set_bbox(dict(facecolor='white', alpha=0.5, edgecolor='none'))
    
    xy = ((pos_in_w1[0] + pos_out[0])/2 - 0.2,(pos_in_w1[1] + pos_out[1])/2 - 0.1)
    t = ax.annotate('$w_1$', xy=xy)
    t.set_bbox(dict(facecolor='white', alpha=0.5, edgecolor='none'))
    
    xy = ((pos_in_w2[0] + pos_out[0])/2 - 0.2,(pos_in_w2[1] + pos_out[1])/2 - 0.15)
    t = ax.annotate('$w_2$', xy=xy)
    t.set_bbox(dict(facecolor='white', alpha=0.5, edgecolor='none'))
    
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim([-1,3])
    ax.set_ylim([-1,3])
    ax.set_aspect('equal','box')

def generateDataNonlinear(n):
    # Generiert Daten in 2D mit 2n Datenpunkten, die mit einer Gerade getrennt werden können
    # Blau oben links und gelb unten rechts.
    
    # Die folgenden vier Zeilen bestimmen die Verteilung der Datenpunkte
    xb = (np.random.rand(n) * 2 - 1) / 2 - 0.5
    yb = (np.random.rand(n) * 2 - 1) / 2 + 2 * (np.random.rand(n) > 0.5)
    xr = (np.random.rand(n) * 2 - 1) / 2 - 0.5
    yr = (np.random.rand(n) * 2 - 1) / 2 + 1
    
    # Der Rest ist nur eine technische Umformung der Daten in das passende Format
    XB = np.stack([xb, yb], axis=1)
    XR = np.stack([xr, yr], axis=1)
    X = np.concatenate([XB, XR])
    #y = np.concatenate([np.zeros(n, dtype=np.int), np.ones(n, dtype=np.int)])
    y = np.concatenate([np.zeros(n, dtype='int'), np.ones(n, dtype='int')])
    return X, y
    
import random
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.linear_model import Perceptron
import numpy as np

# Funktion zum Anzeigen von zwei zufälligen Bildern
def show_random_images():
    global index1, index2
    index1 = random.randint(0, len(digits.images) - 1)
    index2 = random.randint(0, len(digits.images) - 1)
    image1 = digits.images[index1]
    image2 = digits.images[index2]
    combined_image = np.concatenate([image1, image2], axis=1)  # Bilder nebeneinander kombinieren
    ax.matshow(combined_image, cmap='gray')  # Benötigt cmap, da zwei Bilder
    ax.axis('off')
    canvas.draw()
    info_text = f"Indizes: {index1}, {index2}, Zahlen: {digits.target[index1]}, {digits.target[index2]}, Training abgeschlossen. Score: {score:.2f}"
    label.config(text=info_text)

# Funktion zum Neustarten
def restart():
    show_random_images()

# Lade das Datenset
digits = load_digits()

# Perceptron Klassifikation
X, y = load_digits(return_X_y=True)

clf = Perceptron(tol=1e-3, random_state=0)
clf.fit(X, y)
print("Training abgeschlossen.")
score = clf.score(X, y)
print("Score: {:.2f}".format(score))

# Erstelle das GUI-Fenster
root = tk.Tk()
root.title("Zufällige Bilder aus dem Datenset")

# Erstelle das Matplotlib-Figurenfenster
fig = plt.Figure(figsize=(6, 3))
ax = fig.add_subplot(111)
ax.axis('off')

canvas = FigureCanvasTkAgg(fig, master=root)
canvas.get_tk_widget().pack()

# Definiere Index1 und Index2 initial
index1 = 0
index2 = 0

# Erstelle einen Label für die Index- und Zahl-Information
info_text = f"Indizes: {index1}, {index2}, Zahlen: {digits.target[index1]}, {digits.target[index2]}, Training abgeschlossen. Score: {score:.2f}"
label = tk.Label(root, text=info_text)
label.pack()

# Erstelle einen Button zum Neustarten
restart_button = tk.Button(root, text="Neustarten", command=restart)
restart_button.pack()

# Zeige die ersten zufälligen Bilder an
show_random_images()

# Starte die GUI-Schleife
root.mainloop()

Training abgeschlossen.
Score: 0.94


Der hier angegebene Code erzeugt beim Tastenbefehl Shift+Enter ein neues Fenster, welches zwei Zahlen darstellt.
Würdest du auch davon ausgehen das es sich bei den abgebildeten Zahlen um die vom Neuronalen Netzwerk erkannten handelt?